In [1]:
# dataset load
from os.path import join
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

Mounted at /content/drive


In [2]:
import os
import sys

DATA_PATH = '/content/drive/MyDrive/GitHub/DL_Study/datasets'
MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, DATA_PATH)
sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '/content/drive/MyDrive/GitHub/DL_Study/datasets',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [3]:
# import
from config import *
from layers import LSTM, TimeLSTM, FullyConnected
from optim import Adam

# for time series split
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3MB 121kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [4]:
def MSE(y, t):
    return 0.5*np.mean((y-t)**2)

class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x<=0)
        y = x.copy()
        y[self.mask] = 0

        return y
    
    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy

        return dx


class TimeFC:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        N, D = x.shape

        reshaped_x = x.reshape(N, -1)
        y = np.dot(reshaped_x, W) + b
        
        self.x = x
        y = y.reshape(N, -1)
        return y

    def backward(self, dy):
        W, b = self.params
        x = self.x
        N, D = x.shape

        dy = dy.reshape(N, -1)
        reshaped_x = x.reshape(N, -1)

        db = np.sum(dy, axis=0)
        dx = np.matmul(dy, W.T)
        dW = np.matmul(reshaped_x.T, dy)
        
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW
        self.grads[1][...] = db

        return dx

class TimeMSE:
    def __init__(self):
        self.params, self.grads = [], []
        self.activation = ReLU()
        self.cache = None

    def forward(self, xs, ts):
        N, V = xs.shape
        xs = xs.reshape(N, V)
        xs = self.activation.forward(xs)
        ts = ts.reshape(N, V)

        loss = MSE(xs, ts)
        self.cache = (ts, xs, (N, V))

        return loss

    def backward(self, dy = 1):

        ts, xs, (N,  V) = self.cache
        
        dx = dy * (xs - ts) / (N)

        dx = self.activation.backward(dx)
        dx = dx.reshape(N , V)
        return dx

In [5]:
class LstmModelReg:
    def __init__(self, time_size, hidden_size, feature_size):
        T, H, F = time_size, hidden_size, feature_size
        H2 = 64
        rand = np.random.randn

        # weights (Xavier)
        lstm_Wx = (rand(F, 4*H)/ np.sqrt(F)).astype('f')
        lstm_Wh = (rand(H, 4*H)/ np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4*H).astype('f')

        # He initialize
        fc_W1 = (rand(H, H2)/ np.sqrt(H/2)).astype('f')
        fc_b1 = np.zeros(H2).astype('f')

        fc_W2 = (rand(H2, 1)/ np.sqrt(H2/2)).astype('f')
        fc_b2 = np.zeros(1).astype('f')

        # layer
        self.layers = [
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeFC(fc_W1, fc_b1),
            TimeFC(fc_W2, fc_b2)
        ]
        self.loss_layer = TimeMSE()

        self.params, self.grads = [], []

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads


    def predict(self, xs):
        xs = np.array(xs)
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts):
        xs = np.array(xs)
        ts = np.array(ts)
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dy = 1):
        dy = self.loss_layer.backward(dy)
        for layer in reversed(self.layers):
            dy = layer.backward(dy)
        return dy

    def fit(self, train_X=None, train_y=None,learning_rate=0.01, epochs=10, batch_size=32, verbose=0):
        optimizer = Adam(learning_rate)

        data_size = train_X.shape[0]
        max_iters = data_size//batch_size

        for epoch in range(1, epochs+1):
            idx = numpy.random.permutation(numpy.arange(data_size))
            train_X = train_X[idx]
            train_y = train_y[idx]

            epoch_loss = 0
            start_time=time.time()
            
            for iter in range(max_iters):
                batch_x = train_X[iter*batch_size:(iter+1)*batch_size]
                batch_y = train_y[iter*batch_size:(iter+1)*batch_size]

                loss = self.forward(batch_x, batch_y)
                self.backward()
                params, grads = self.params, self.grads
                optimizer.update(params, grads)

                epoch_loss += loss
            avg_loss = epoch_loss/max_iters

            if verbose:
                duration = start_time-time.time()
                print(f'epoch:{epoch}/{epochs}, 시간:{duration:.2f}[s], loss:{avg_loss:.5f}')


    def reset_state(self):
        self.layers[0].reset_state()

In [6]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [32, 64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = LstmModelReg(time_size=24, hidden_size=100, feature_size=12)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(np.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [7]:
# dataset
import zipfile, requests, io
import pandas as pd
import numpy
import time
from datetime import datetime

numpy.random.seed(42)
np.random.seed(42)

df_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H.%M.%S')
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip'
r = requests.get(data_url)
with zipfile.ZipFile(io.BytesIO(r.content)) as z:
    with z.open('AirQualityUCI.csv') as f:
        df = pd.read_csv(f, sep=';')

df = df.iloc[:9357]
df.drop(['Date', 'Time', 'Unnamed: 15', 'Unnamed: 16'], axis=1, inplace=True)
df = df.astype(str)
df.replace('(.*),(.*)', r'\1.\2', regex=True, inplace=True)
df = df.astype('float')
df.replace(-200, numpy.nan, inplace=True)       # -200 is missing value
df.drop(['NMHC(GT)'], axis=1, inplace=True)     # almost NaN value
df.ffill(axis=0, inplace=True)                  # forward fill for missing value
df.head()

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2.6,1360.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,2.0,1292.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.2,1402.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1376.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,1.6,1272.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [8]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

cols = df.columns.tolist()
cols = cols[1:2]+cols[0:1]+cols[2:]
dataset = df[cols]
dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 12
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)


n_splits = 3
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []
best_error = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)
    error = np.sqrt(MSE_metric(predicted, test_cv_y))
    best_error.append(error)

    # model eval
    print(f'best_model => error(rmse) : {error}, param:{configs[param]}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/3
 == train [30, 32, 0.01] model ==   error(RMSE):0.7343853649416153
 == train [30, 32, 0.001] model ==   error(RMSE):0.6213059022576115
 == train [30, 64, 0.01] model ==   error(RMSE):0.5751691362949465
 == train [30, 64, 0.001] model ==   error(RMSE):0.6529860531068352
best_model => error(rmse) : 0.6965777572907911, param:[30, 64, 0.01]

fold : 2/3
 == train [30, 32, 0.01] model ==   error(RMSE):0.6800323761201444
 == train [30, 32, 0.001] model ==   error(RMSE):0.6605055000389755
 == train [30, 64, 0.01] model ==   error(RMSE):0.6845942311454192
 == train [30, 64, 0.001] model ==   error(RMSE):3.1995735809490746
best_model => error(rmse) : 0.8201651594838518, param:[30, 32, 0.001]

fold : 3/3
 == train [30, 32, 0.01] model ==   error(RMSE):0.8377379244799682
 == train [30, 32, 0.001] model ==   error(RMSE):0.7931917653655889
 == train [30, 64, 0.01] model ==   error(RMSE):0.8081571748736346
 == train [30, 64, 0.001] model ==   erro

In [9]:
model_evaluation = sum(best_error)
model_evaluation /= n_splits
print(f'evaluation [Mean RMSE] : {model_evaluation}')

selected_model.reset_state()
predicted = selected_model.predict(test_cv_X)
if GPU:
    predicted = np.asnumpy(predicted)
print(f'MSE : {MSE_metric(predicted, test_cv_y)}')
print(f'RMSE : {np.sqrt(MSE_metric(predicted, test_cv_y))}')

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))
print(f'MAE : {MAE_metric(predicted, test_cv_y)}')

evaluation [Mean RMSE] : 0.7555788935278569
MSE : 0.5624906457522814
RMSE : 0.7499937638089276
MAE : 0.5271304300176689
